<a href="https://colab.research.google.com/github/nisha-s10/Deep-Learning-Lab-AFI524/blob/main/Experiment%204%5CExperiment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 1: Imports & Setup

In [8]:
import os, shutil, zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

# STEP 2: Extract Cats vs Dogs Dataset

In [9]:
with zipfile.ZipFile("/content/train.zip", "r") as zip_ref:
    zip_ref.extractall("/content/cats_dogs")

# STEP 3: Fix Kaggle Folder Structure

In [10]:
train_dir = "/content/cats_dogs/train"
os.makedirs(f"{train_dir}/cat", exist_ok=True)
os.makedirs(f"{train_dir}/dog", exist_ok=True)

for f in os.listdir(train_dir):
    fp = os.path.join(train_dir, f)
    if os.path.isdir(fp):
        continue
    if f.startswith("cat"):
        shutil.move(fp, f"{train_dir}/cat/{f}")
    elif f.startswith("dog"):
        shutil.move(fp, f"{train_dir}/dog/{f}")

# STEP 4: Transforms (Preprocessing)

In [11]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

# STEP 5: Load Cats vs Dogs Dataset

In [12]:
catsdogs_dataset = datasets.ImageFolder(
    root="/content/cats_dogs/train",
    transform=transform
)

train_size = int(0.8 * len(catsdogs_dataset))
val_size = len(catsdogs_dataset) - train_size

cd_train, cd_val = random_split(catsdogs_dataset, [train_size, val_size])

cd_train_loader = DataLoader(cd_train, batch_size=32, shuffle=True)
cd_val_loader   = DataLoader(cd_val, batch_size=32, shuffle=False)

print("Classes:", catsdogs_dataset.classes)

Classes: ['cat', 'dog']


# STEP 6: Load CIFAR-10 Dataset

In [13]:
cifar_train = datasets.CIFAR10(
    root="./cifar10",
    train=True,
    download=True,
    transform=transform
)

cifar_test = datasets.CIFAR10(
    root="./cifar10",
    train=False,
    download=True,
    transform=transform
)

cifar_train_loader = DataLoader(cifar_train, batch_size=32, shuffle=True)
cifar_test_loader  = DataLoader(cifar_test, batch_size=32, shuffle=False)

100%|██████████| 170M/170M [00:02<00:00, 73.8MB/s]


# STEP 7: CNN Model (Configurable)

In [14]:
class SimpleCNN(nn.Module):
    def __init__(self, activation, num_classes):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            activation(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            activation(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            activation(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# STEP 8: Weight Initialization

In [15]:
def init_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            if init_type == "xavier":
                nn.init.xavier_uniform_(m.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(m.weight, nonlinearity="relu")
            else:
                nn.init.normal_(m.weight, 0, 0.01)


# STEP 9: Optimizer Selector

In [16]:
def get_optimizer(name, model):
    if name == "sgd":
        return optim.SGD(model.parameters(), lr=0.01)
    elif name == "adam":
        return optim.Adam(model.parameters(), lr=0.001)
    else:
        return optim.RMSprop(model.parameters(), lr=0.001)

# STEP 10: Training & Evaluation Loop

In [17]:
def train_model(model, train_loader, val_loader, optimizer, epochs=3):
    criterion = nn.CrossEntropyLoss()
    best_acc = 0

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                preds = model(x).argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        best_acc = max(best_acc, acc)
        print(f"Epoch {epoch+1}, Val Accuracy: {acc:.4f}")

    return best_acc

# STEP 11: Run ALL CNN Experiments (3×3×3) for Cat-Dog

In [ ]:
activations = {
    "relu": nn.ReLU,
    "tanh": nn.Tanh,
    "leaky": nn.LeakyReLU
}

inits = ["random", "xavier", "kaiming"]
optimizers = ["sgd", "adam", "rmsprop"]

best_cd_acc = 0
best_cd_model = None

for act in activations:
    for init in inits:
        for opt in optimizers:
            print(f"\nCats vs Dogs → {act}, {init}, {opt}")

            model = SimpleCNN(activations[act], num_classes=2)
            init_weights(model, init)
            optimizer = get_optimizer(opt, model)

            acc = train_model(
                model,
                cd_train_loader,
                cd_val_loader,
                optimizer
            )

            if acc > best_cd_acc:
                best_cd_acc = acc
                best_cd_model = model


Cats vs Dogs → relu, random, sgd
Epoch 1, Val Accuracy: 0.6882


# STEP 12: Save Best CNN Models

In [ ]:
torch.save(best_cd_model.state_dict(), "best_cnn_catsdogs.pth")
print("Best Cats vs Dogs CNN saved")

# STEP 13: ResNet-18 (Transfer Learning)

In [ ]:
resnet = models.resnet18(pretrained=True)

for p in resnet.parameters():
    p.requires_grad = False

resnet.fc = nn.Linear(resnet.fc.in_features, 2)

optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

train_model(
    resnet,
    cd_train_loader,
    cd_val_loader,
    optimizer
)

torch.save(resnet.state_dict(), "resnet18_catsdogs.pth")

# STEP 14: Run ALL CNN Experiments (3×3×3) for Cifar10

In [ ]:
best_cifar_acc = 0
best_cifar_model = None
best_cifar_config = None

for act in activations:
    for init in inits:
        for opt in optimizers:
            print(f"\nCIFAR-10 → {act}, {init}, {opt}")

            model = SimpleCNN(activations[act], num_classes=10)
            init_weights(model, init)
            optimizer = get_optimizer(opt, model)

            acc = train_model(
                model,
                cifar_train_loader,
                cifar_test_loader,
                optimizer,
                epochs=3   # keep small for Colab
            )

            if acc > best_cifar_acc:
                best_cifar_acc = acc
                best_cifar_model = model
                best_cifar_config = (act, init, opt)

# STEP 15: Save Best Model

In [ ]:
torch.save(best_cifar_model.state_dict(), "best_cnn_cifar10.pth")
print("Best CIFAR-10 CNN saved")
print("Best CIFAR-10 config:", best_cifar_config)

# STEP 16: ResNet-18


In [ ]:
resnet_cifar = models.resnet18(pretrained=True)

for param in resnet_cifar.parameters():
    param.requires_grad = False

resnet_cifar.fc = nn.Linear(resnet_cifar.fc.in_features, 10)

optimizer = optim.Adam(resnet_cifar.fc.parameters(), lr=0.001)

train_model(
    resnet_cifar,
    cifar_train_loader,
    cifar_test_loader,
    optimizer,
    epochs=3
)

torch.save(resnet_cifar.state_dict(), "resnet18_cifar10.pth")